This script calculates an airfoil and outputs it to a format to be read by gmsh works for gmsh 4.7.1

In [ ]:
#import common modules
import numpy as np
import matplotlib.pyplot as plt
import cv2

from pathlib import Path

In [ ]:
img_location = Path('./modifiedFoil.png')

img2 = cv2.imread(img_location.as_posix(), cv2.IMREAD_COLOR)
img = cv2.imread(img_location.as_posix(), cv2.IMREAD_GRAYSCALE)
# Converting image to a binary image
# ( black and white only image).
_, threshold = cv2.threshold(img, 110, 255, cv2.THRESH_BINARY)
  
# Detecting contours in image.
cnts, _= cv2.findContours(threshold, cv2.RETR_TREE,
                               cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
# create a blank image
mask = np.zeros(img.shape, dtype=np.uint8)
# find the pixel points
cv2.drawContours(mask, cnts, 1, 255, 1)
# Get the contour points
pixelPoints = np.transpose(np.nonzero(mask))
hullX = pixelPoints[:,1]
hullY = pixelPoints[:,0]

# perform some shifting to get the original object
hullX = hullX - np.min(hullX)
hullY = hullY - (np.max(hullY) + np.min(hullY))/2

length_ = 1.0
height_ = 0.06

hullX = hullX * (length_ / np.max(hullX))
hullY = hullY * (height_ / np.max(hullY))

sort_ind = np.argsort(hullX, axis=0)
hullX = hullX[sort_ind]
hullY = hullY[sort_ind]

fig3, ax3 = plt.subplots()
ax3.plot(hullX, hullY, 'o')
ax3.axis('equal')

In [ ]:
# now we sample and condition the data so we can put it in gmsh
coord_points = 50

yu_indices = np.where(hullY >= 0)[0]

yu_fit = np.polyfit(hullX[yu_indices], hullY[yu_indices], 8)
yu_poly = np.poly1d(yu_fit)

beta = np.linspace(0, np.pi, coord_points)
xu = (0.5*(1-np.cos(beta)))
xl = xu
yu = yu_poly(xu)
yl = -yu_poly(xu)


We can make things easier by defining a characteristic length for the mesh and a fluid domain based on the geometry

In this case:

all variables are a factor of total length e.g. 40 is 40 * length of airfoil

Lx_plus = length behind the airfoil

Lx_minus = length in front of airfoil

Ly = length top and bottom of airfoil

No we should calculate the y+ value and all our boundary layer values

Credit to https://www.fluidmechanics101.com/pdf/calculators.pdf

For the calculation processes


In [ ]:
# Define our setup variables
U = 1.0                                 # Freestream Velocity                                 # Length            
density = 1025                          # fluid density
kinVisc = 1e-8                       # kinematic viscosity of fluid
dynamicVisc = density * kinVisc         # dynamic viscosity of fluid
yPlus = 5.0                             # desired Y+
growth_ratio = 1.1                      # growth ratio
N = 30                                  # number of inflation layers

# calculate the Reynolds Number
Re = U * length_ / kinVisc

# calculate the skin friction coefficient
Cf = (2*np.log10(Re) - 0.65)**(-2.3)

# calculate the shear wall stress
tw = 0.5 * density * U**2 * Cf

# calculate the friction velocity
ut = np.sqrt(tw / density)

# calculate yp where yp is the distance to the centroid of the first cell
# based on desired y+. yH is the first cell height which is 2 times the centroid
yp =  (yPlus * dynamicVisc) / (ut * density)

yH = 2 * yp

# Estimate the boundary layer thickness using the Blasius solution
# laminar and turbulent are given here
if ( Re < 4e5):
    delta = 4.91 * length_ / np.sqrt(Re)
else:
    delta = 0.38 * length_ / Re**(1/5)

# calculate the total thickness of the inflation layers yT
# specify the growth ratio
yT = yH * ((1 - growth_ratio**N)/(1 - growth_ratio))

#f = lambda r: (1.14e-8 * ((1 - r**20)/(1 - r)) - 0.00288)

#growth_ratio_calc = optimize.fsolve(f, 1.1)
# Give the final layer thickness
yFinal = yH * growth_ratio**(N - 1)

print(yH, growth_ratio, yFinal)

We can make things easier by defining a characteristic length for the mesh and a fluid domain based on the geometry

In this case:

all variables are a factor of total length e.g. 40 is 40 * length of airfoil

Lx_plus = length behind the airfoil

Lx_minus = length in front of airfoil

Ly = length top and bottom of airfoil


In [ ]:
airfoil_cl = 0.005
bl_cl = 0.005

hfar = 0.0001
hwall_n = 0.00005
thickness = 0.01  # thickness of boundary layer

In [ ]:
# Create array for writing
point = np.zeros((2*coord_points-2,3))
ind = 0
back_plane_z = -0.5
bLength = 2

point[0:coord_points, 0] = xu
point[0:coord_points, 1] = yu
point[0:coord_points, 2] = back_plane_z

point[coord_points:, 0] = np.flipud(xl[1:-1])
point[coord_points:, 1] = np.flipud(yl[1:-1])
point[coord_points:, 2] = back_plane_z

num_points = point.shape[0]
airfoil_point_numbers = np.arange(1, num_points, 1)
num_lines = num_points;

# perform some conditioning on the 0 coordinate and max length coordinate 
# so that the b-spline connects better
point[np.where(point[:,0] == length_), 1] = 0.0
point[np.where(point[:,0] == 0.0), 1] = 0.0

point2 = np.copy(point)
point2[:,0] = point2[:,0] * bLength - (xl[-1]*0.33)
point2[:,1] = point2[:,1] * bLength

airfoil_point_index = np.arange(1, point.shape[0]+1)
airfoil_bl_point_index = np.arange(airfoil_point_index[-1]+1, airfoil_point_index[-1]+1+point2.shape[0])

In [ ]:
#Write data to the .geo file
with open('modified_small.geo', 'w') as df:
    # first we write the variables for easy changing locally later
    df.write('airfoil_cl = %f;\n' % (airfoil_cl))
    df.write('bl_cl = %f;\n' % (bl_cl))
    df.write('\n')

    for i, index in zip(np.arange(0, point.shape[0], 1), airfoil_point_index):
        df.write('Point(%i) = {%f, %f, %f, airfoil_cl};\n' % (index, point[i,0], point[i,1],
                                                    point[i,2]))
    point_index = i
    df.write('\n')

    for i, index in zip(np.arange(0, point2.shape[0], 1), airfoil_bl_point_index):
        df.write('Point(%i) = {%f, %f, %f, bl_cl};\n' % (index, point2[i,0], point2[i,1],
                                                    point2[i,2]))
        point_index += 1
    df.write('\n')
    
    # define the domain point numbers and domain line numbers
    line_loops = np.array([101, 102])
    plane_surface = np.array([201])
    
    # write the boundary domain points
    df.write('// combine foil points into a bspline \n')
    df.write('BSpline(5) = {'  + str(airfoil_point_index[0]) + ':' + str(airfoil_point_index[int(airfoil_point_index.shape[0]/2)]) + '}; \n')
    df.write('BSpline(6) = {' + str(airfoil_point_index[int(airfoil_point_index.shape[0]/2)]) + ':' + str(airfoil_point_index[-1]) + ',' +  str(airfoil_point_index[0]) + '}; \n')
    df.write('Line Loop(' + str(line_loops[0]) + ') = {5, 6}; \n') 
    df.write('\n')
       
    df.write('\n')
    df.write('// combine foil points into a bspline \n')
    df.write('BSpline(7) = {'  + str(airfoil_bl_point_index[0]) + ':' + str(airfoil_bl_point_index[int(airfoil_bl_point_index.shape[0]/2)]) + '}; \n')
    df.write('BSpline(8) = {' + str(airfoil_bl_point_index[int(airfoil_bl_point_index.shape[0]/2)]) + ':' + str(airfoil_bl_point_index[-1]) + ',' +  str(airfoil_bl_point_index[0]) + '}; \n')
    df.write('Line Loop(' + str(line_loops[1]) + ') = {7, 8}; \n') 
    df.write('\n')
   
    df.write('Plane Surface(%i) = {101, 102}; \n' % (plane_surface[0]))
    
    df.write('\n')
    # extrude the surface so we can name the boundaries
    df.write('Extrude {0, 0, 1} {\n Surface{%i};\n Layers{1};\n Recombine;\n}' % 
    (plane_surface[0]))
    df.write('\n')

    # # define the physical surface patches
    # df.write('// Define the physical surfaces -> values are obtained via the GUI \n')
    # df.write('Physical Surface("overPatch") = {212, 216, 220, 224}; \n')
    # df.write('Physical Surface("symFront") = {233}; \n')
    # df.write('Physical Surface("symBack") = {201}; \n')
    # df.write('Physical Surface("wingT") = {232}; \n')
    # df.write('Physical Surface("wingB") = {228}; \n')
    # df.write('\n')

    # define the internal volume
    df.write('// Define the internal volume - usually 1 but obtained from GUI \n')
    df.write('Physical Volume("internal") = {1};\n')
    df.write('\n')

    # define the boundary layer addition
    df.write('//Define the boundary layer \n')
    df.write('Field[1] = BoundaryLayer; \n')
    df.write('Field[1].EdgesList = {5, 6}; \n')
    df.write('Field[1].NodesList = {1, 1021, 1022, ' + str(airfoil_point_numbers[-1]) + '}; \n')
    df.write('Field[1].hfar = ' + str(hfar) + '; \n')
    df.write('Field[1].hwall_n = ' + str(hwall_n) + '; \n')
    df.write('Field[1].thickness = ' + str(thickness) + '; \n')
    df.write('Field[1].ratio = ' + str(growth_ratio) + '; \n')
    df.write('Field[1].AnisoMax = 10; \n')
    df.write('Field[1].Quads = 1; \n')
    df.write('Field[1].IntersectMetrics = 0; \n')
    df.write('BoundaryLayer Field = 1; \n')

In [ ]:
print(point[:,2], point2[:,2])